In [38]:
model_name = "t5-small"
model_path = "../Model Data/t5-small/t5-small_001/"

# Libraries installation

Install Transformers libaries, framework seqeval, some extensions: unidecode for formatting accent language, datasets for creating a Dataset,...

In [1]:
# #Install required libraries
# !pip install datasets transformers evaluate seqeval unidecode google.colab
# !pip install transformers[torch]

## WandB

In [2]:
# !pip install wandb
# !wandb login

In [3]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py

--2023-12-11 20:23:33--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/pytorch/text-classification/run_glue.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 28335 (28K) [text/plain]
Saving to: ‘run_glue.py.2’

run_glue.py.2       100%[===================>]  27.67K  --.-KB/s    in 0.01s   

2023-12-11 20:23:34 (2.24 MB/s) - ‘run_glue.py.2’ saved [28335/28335]



In [6]:
# !pip install -q git+https://github.com/huggingface/transformers

In [7]:
# import wandb

In [8]:
# wandb.login()

In [9]:
# %env WANDB_LOG_MODEL=true

## Import required libraries

In [4]:
#Import libraries to project
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
import torch
from datasets import Dataset
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
import json
import pandas as pd
from unidecode import unidecode
import os
import logging

/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Mount Google Drive to Google Colab

In [5]:
# #Mount Drive to Colab
# from google.colab import drive
# drive.mount('/content/gdrive')

# Data preparation

Read training set from dataset stored in Google Drive.
Create a list of input and output sequences

In [6]:
#Read datasets

root_data_dir = "../Datasets/NER/BiodivNER"

dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [7]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="utf-8")
  data = data.fillna(method="ffill")
  return data

In [8]:
data = loadData(train_csv_file_path)
val_data = loadData(val_csv_file_path)
test_data = loadData(test_csv_file_path)

/tmp/ipykernel_393286/1497070395.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")
/tmp/ipykernel_393286/1497070395.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")
/tmp/ipykernel_393286/1497070395.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


In [9]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [10]:
getter = SentenceGetter(data)
sentences = getter.sentences
sent = getter.get_next()
print(sent)

[('bottom', 'O'), ('board', 'O'), ('debris', 'O'), (',', 'O'), ('frames', 'O'), (',', 'O'), ('landing', 'O'), ('platforms', 'O'), (')', 'O'), (',', 'O'), ('and', 'O'), ('isolates', 'O'), ('of', 'O'), ('microbes', 'O'), (',', 'O'), ('parasites', 'O'), ('and', 'O'), ('pathogens', 'O'), ('from', 'O'), ('honey', 'B-Organism'), ('bees', 'I-Organism'), ('.', 'O')]


In [11]:
getter_val = SentenceGetter(val_data)
sentences_val = getter_val.sentences
sent_val = getter_val.get_next()
print(sent_val)

[('However', 'O'), (',', 'O'), ('the', 'O'), ('lack', 'O'), ('of', 'O'), ('correlation', 'O'), ('between', 'O'), ('dung', 'O'), ('beetle', 'O'), ('community', 'B-Environment'), ('characteristics', 'O'), ('and', 'O'), ('dung', 'O'), ('removal', 'O'), ('highlights', 'O'), ('the', 'O'), ('need', 'O'), ('for', 'O'), ('further', 'O'), ('research', 'O'), ('into', 'O'), ('spatial', 'O'), ('variation', 'O'), ('in', 'O'), ('biodiversityÃ¢â\x82¬â\x80\x9cecosystem', 'O'), ('function', 'O'), ('relationships', 'O'), ('and', 'O'), ('how', 'O'), ('the', 'O'), ('results', 'O'), ('of', 'O'), ('such', 'O'), ('studies', 'O'), ('are', 'O'), ('affected', 'O'), ('by', 'O'), ('methodological', 'O'), ('choices', 'O'), ('.', 'O')]


In [12]:
getter_test = SentenceGetter(test_data)
sentences_test = getter_test.sentences
sent_test = getter_test.get_next()
print(sent_test)


[('The', 'O'), ('primacy', 'O'), ('of', 'O'), ('either', 'O'), ('species', 'B-Quality'), ('or', 'O'), ('functional', 'O'), ('group', 'O'), ('richness', 'B-Quality'), ('effects', 'O'), ('depended', 'O'), ('on', 'O'), ('the', 'O'), ('sequence', 'O'), ('of', 'O'), ('testing', 'O'), ('these', 'O'), ('terms', 'O'), (',', 'O'), ('indicating', 'O'), ('that', 'O'), ('both', 'O'), ('aspects', 'O'), ('of', 'O'), ('richness', 'B-Quality'), ('were', 'O'), ('congruent', 'O'), ('and', 'O'), ('complementary', 'O'), ('to', 'O'), ('expected', 'O'), ('strong', 'O'), ('effects', 'O'), ('of', 'O'), ('legume', 'O'), ('presence', 'O'), ('and', 'O'), ('grass', 'B-Organism'), ('presence', 'O'), ('on', 'O'), ('plant', 'B-Organism'), ('chemical', 'B-Quality'), ('composition', 'I-Quality'), ('.', 'O')]


In [13]:
def get_text_tags_lists(sentences):
  texts = []
  tags = []
  for sent in sentences: #list of tuples
    sent_texts = []
    sent_tags = []
    for tuple1 in sent:
      sent_texts.append(tuple1[0])
      sent_tags.append(tuple1[1])

    texts.append(sent_texts)
    tags.append(sent_tags)
  return texts, tags

In [14]:
train_texts, train_tags = get_text_tags_lists(sentences)
val_texts, val_tags = get_text_tags_lists(sentences_val)
test_texts, test_tags = get_text_tags_lists(sentences_test)

In [15]:
#Create a list of texts
def create_input_seq(texts):
  inputs = texts
  input_sequences = []

  for sentence in inputs:
    input_sequences.append(' '.join(sentence))

  return input_sequences

In [16]:
train_inputs_raw = create_input_seq(train_texts)
val_inputs_raw = create_input_seq(val_texts)
test_inputs_raw = create_input_seq(test_texts)

Convert problem into text-to-text format by formatting an output which is a sequence of class:entity pairs separated by a semicolon ';'

In [17]:
def format_targets(text, label):
  s = ""
  texts = []
  for i in range(len(text)-1):
    if label[i] != 'O':
      s += text[i] + " "
      if label[i+1] == 'O' or label[i+1][0:1] == 'B':
        s = label[i][2:] + ": " + s
        texts.append(s.strip())
        s = ""
  texts = "; ".join(texts)
  return texts

In [18]:
str1 = format_targets(train_texts[0], train_tags[0])
str1

'Quality: Density; Phenomena: Treatment; Quality: Dead; Environment: Garden; Organism: Seedling; Phenomena: growth; Quality: biomass; Environment: plant communities; Quality: species richness; Environment: tropical forest ecosystems'

 Convert them back to sentence format

In [19]:
def create_output_seq(texts, tags):
  output_sequences = []

  for i in range(len(texts)):
    text = texts[i]
    label = tags[i]

    target = format_targets(text, label)
    output_sequences.append(target)

  return output_sequences

In [20]:
train_outputs_raw = create_output_seq(train_texts, train_tags)
val_outputs_raw = create_output_seq(val_texts, val_tags)
test_outputs_raw = create_output_seq(test_texts, test_tags)

In [21]:
def has_non_ascii(sentence):
    return any(ord(char) > 127 for char in sentence)

def clean_non_ascii(inputs, labels):
    clean_inputs = []
    clean_labels = []

    for sentence, label in zip(inputs, labels):
        if has_non_ascii(sentence):
            continue
        
        clean_inputs.append(sentence)
        clean_labels.append(label)

    return clean_inputs, clean_labels

def clean_non_ascii_test(texts, tags):
    clean_inputs = []
    clean_labels = []

    for sentence, label in zip(texts, tags):
        sentence2 = ' '.join(sentence)
        if has_non_ascii(sentence2):
            continue
        
        clean_inputs.append(sentence)
        clean_labels.append(label)

    return clean_inputs, clean_labels

In [22]:
train_inputs, train_outputs = clean_non_ascii(train_inputs_raw, train_outputs_raw)
val_inputs, val_outputs = clean_non_ascii(val_inputs_raw, val_outputs_raw)
test_texts_c, test_tags_c = clean_non_ascii_test(test_texts, test_tags)

In [23]:
print(test_texts_c)

[['Combining', 'metabolic', 'and', 'food-', 'web', 'theory', ',', 'we', 'calculate', 'annual', 'energy', 'fluxes', 'to', 'model', 'impacts', 'of', 'land-use', 'intensification', 'on', 'multitrophic', 'ecosystem', 'functioning', '.'], ['The', 'primacy', 'of', 'either', 'species', 'or', 'functional', 'group', 'richness', 'effects', 'depended', 'on', 'the', 'sequence', 'of', 'testing', 'these', 'terms', ',', 'indicating', 'that', 'both', 'aspects', 'of', 'richness', 'were', 'congruent', 'and', 'complementary', 'to', 'expected', 'strong', 'effects', 'of', 'legume', 'presence', 'and', 'grass', 'presence', 'on', 'plant', 'chemical', 'composition', '.'], ['Full', 'ant', 'species', 'name', '.'], ['If', 'it', 'would', 'not', 'be', 'considered', 'greater', 'than', '10', ',', 'it', 'would', 'not', 'be', 'considered', 'in', 'the', 'outer', 'subplots.Helper', ';', 'Datagroup', 'description', ':', 'Helper', ')', 'dimensionless', 'real', 'Helper', 'Helper', 'column', 'to', 'understand', 'other', 'col

In [24]:
for row in train_outputs[:10]:
  print(row, "\n")

Quality: Density; Phenomena: Treatment; Quality: Dead; Environment: Garden; Organism: Seedling; Phenomena: growth; Quality: biomass; Environment: plant communities; Quality: species richness; Environment: tropical forest ecosystems 

Organism: honey bees 

Environment: soil 

Environment: riparian reserves; Environment: forest; Environment: riparian reserves; Matter: oil palm 

Organism: species 

Phenomena: precipitation; Quality: rainfall amount; Environment: field; Phenomena: rainfall; Quality: rainfall amount; Environment: field; Quality: average rainfall intensity; Environment: field; Quality: peak rainfall intensity; Environment: field; Quality: intensities; Environment: field; Quality: rainfall amount; Quality: average rainfall intensity; Quality: peak rainfall intensity; Quality: intensities; Environment: field; Quality: intensities; Phenomena: Precipitation; Phenomena: Precipitation 

Environment: forest; Environment: tropical forest landscape; Quality: area; Environment: cano

In [25]:
train_inputs[11], train_outputs[11]

('-- - 6 digit metal tags starting with 3 were also used for woody debris items -- the reference file linking metal tags to tree individual tags is called `` Tree stem reference list for the Comparative Study Sites ( CSPs ) ) ( neigh.label : label ID of neighbouring individual : We locate every tagged individual CWD piece .',
 'Matter: woody debris items')

# Training preparation

We first load a ViT5 tokenizer and a pretrained model powered by Transformers library

In [26]:
#Create a tokenizer and a model supported by ViT5
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
# model.to("cuda")

Define a function which converts data to tokens

In [27]:
#Function for tokenizing texts and labels
def preprocess(examples):
  model_inputs = tokenizer(
      examples["inputs"], max_length=512, truncation=True, padding=True
  )
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(
        examples["labels"], max_length=512, truncation=True, padding=True
    )
  model_inputs["labels"] = labels['input_ids']
  model_inputs['input_ids'] = model_inputs['input_ids']
  return model_inputs

Define a dictionary that holds inputs and their corresponding labels

Create a dataset from that dictionary and apply the tokenization process to every data sample with map function

In [28]:
#Tokenise dataset
def dict_to_dataset(inputs, outputs):
  dict_obj = {'inputs':inputs, 'labels':outputs}
  dataset = Dataset.from_dict(dict_obj)
  tokenized_datasets = dataset.map(preprocess, batched=True, num_proc=8)

  return tokenized_datasets

In [29]:
train_tokenized = dict_to_dataset(train_inputs, train_outputs)
val_tokenized = dict_to_dataset(val_inputs, val_outputs)
# test_tokenized = dict_to_dataset(test_inputs, test_outputs)

Map (num_proc=8):   0%|          | 0/1776 [00:00<?, ? examples/s]/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3856: U

Map (num_proc=8):   0%|          | 0/215 [00:00<?, ? examples/s]/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map (num_proc=8):  13%|█▎        | 27/215 [00:00<00:00, 197.42 examples/s]/home/nlpbiodiv2023/.local/lib

In [30]:
logging_dir="./log"
logging_dir_name="training_logs"
logging_dir_path = os.path.join(logging_dir, logging_dir_name)
os.makedirs(logging_dir_path, exist_ok=True)

In [31]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [32]:
from seqeval import metrics as seqeval_metrics

In [33]:
def compute_metrics(p):
  predictions, labels = p.predictions, p.label_ids
  flat_predictions = [label for sentence_labels in predictions for label in sentence_labels]
  flat_labels = [label for sentence_labels in labels for label in sentence_labels]
  accuracy = seqeval_metrics.accuracy_score([flat_labels], [flat_predictions])
  return {"accuracy": accuracy}

Set training arguments for training process

In [34]:
#Arguments for training model
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")
training_args = Seq2SeqTrainingArguments('../Model Data/t5-small/t5-small_001',
                                      report_to = 'wandb',
                                      do_train=True,
                                      do_eval=True,
                                      num_train_epochs=10,
                                      learning_rate=2e-5,
                                      warmup_ratio=0.00,
                                      weight_decay=0.01,
                                      per_device_train_batch_size=4,
                                      per_device_eval_batch_size=4,

                                      logging_steps=50,
                                      group_by_length=True,
                                      # save_strategy="epoch",
                                      save_total_limit=0,
                                      #eval_steps=1,
                                      evaluation_strategy="epoch",
                                      # evaluation_strategy="no",
                                      # fp16=True,
                                      # eval_accumulation_steps=50
                                      )

Make sure to push the model to GPU for optimisation

In [35]:
model.to("cuda" if torch.cuda.is_available() else "cpu")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

# Train Named Entity Recognition model

In [36]:
# #Procession of training model
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shannen (shannen-team). Use `wandb login --relogin` to force relogin


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,0.393700,0.670642
2,0.165600,0.428507
3,0.122600,0.330167
4,0.108700,0.274708
5,0.089900,0.228207
6,0.089800,0.198058
7,0.073100,0.176359
8,0.084800,0.167968
9,0.067700,0.164274
10,0.065400,0.162634


/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along

TrainOutput(global_step=2220, training_loss=0.41993976681082096, metrics={'train_runtime': 938.4612, 'train_samples_per_second': 18.925, 'train_steps_per_second': 2.366, 'total_flos': 2403670394142720.0, 'train_loss': 0.41993976681082096, 'epoch': 10.0})

In [51]:
trainer.evaluate(eval_dataset=val_tokenized)

/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.3359127938747406,
 'eval_runtime': 3.2573,
 'eval_samples_per_second': 66.006,
 'eval_steps_per_second': 8.289,
 'epoch': 5.0}

In [39]:
# Assuming that 'model' is your Seq2Seq model
model.save_pretrained(os.path.join(model_path, "final"))

# Inference

Load fine-tuned model stored in Google Drive with a Tokenizer by ViT5

In [45]:
#Load model saved as a checkpoint
from transformers import AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained('t5-small')
# model = AutoModelForSeq2SeqLM.from_pretrained("../Model Data/t5-small/t5-small_001/checkpoint-1000")
model = trainer.model
# model.to("cuda")

In [46]:
#Total number of trainable parameters
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

60506624

In [52]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

To use the model at its best, make sure the data fed to the model is word-level,
 if it's not, simply use an RDR-Segmenter!

The cell below helps produce target text with 'generate' method

In [53]:
sentence = train_inputs[100]
# sentence = eval_input_sequences[23]
encoding = tokenizer(sentence, return_tensors="pt", max_length=512)
input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
input_ids

tensor([[ 1636,    37, 21965,  7322,    19,   787,    57,   309,  1713,  3316,
           448,  2990,    55,  4505,     3,   102,   834,    23,     2,   357,
             3,     6,    28,     3,   102,   834,    23,  9085,     8,  5237,
         15025,    13,     8,    34,   107,  3244,     3,     5,     1]])

In [56]:
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=512,
)

KeyboardInterrupt: 

In [55]:
print(outputs[0])

NameError: name 'outputs' is not defined

In [57]:
labels = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Results: ", labels)
print("Actual labels:", train_outputs[50])

NameError: name 'outputs' is not defined

In [58]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

'word_labels' takes into account two parameters 'sentence' and 'labels' to calculate the correct BIO label and map them to a list

In [59]:
def word_labels(sentence, labels):
  predictions = ["O" for i in range(len(sentence.split()))]
  if labels != '':
    list_labels = labels.split(";")
    sent = sentence.split()

    start = 0
    for i in range(len(list_labels)):
      sub_list = list_labels[i].split(":")
      # print(list_labels[i])
      # print(sub_list)
      class_entity = sub_list[0].strip() # location, organization, age,...
      named_entity = sub_list[1].strip().lower() # Ha Noi, London, 433,...
      named_entity_element = named_entity.split() # Ha, Noi, London, 433, Soc Trang,...

      flist = []
      for i in range(len(named_entity_element)):
        if named_entity_element[i][-1] == ',':
          entity1 = named_entity_element[i][0:len(named_entity_element[i])-1]
          entity2 = ","
          flist.append(entity1)
          flist.append(entity2)
        else:
          flist.append(named_entity_element[i])

      named_entity_element = flist
      for i in range(len(named_entity_element)):
        try:
          findex = sent.index(named_entity_element[i], start)
          start = findex + 1
          f_class = ""
          if i == 0:
            f_class = "B-" + class_entity
          else:
            f_class = "I-" + class_entity
          predictions[findex] = f_class
        except:
          pass

  return predictions

# reverse the subword tokens to their real words

Load dataset for devaluation

In [60]:
# #Read test set
# evalWord = [json.loads(line) for line in open('gdrive/MyDrive/Ct550/word/test_word.json', 'r', encoding='utf-8')]
# evalWordData = pd.DataFrame(evalWord)
# evalWordData = evalWordData.rename(columns={'tags':'target_text', 'words':'source_text'})

Create lists of input and output sequences

In [61]:
# #Create a list of texts
eval_input_data = test_texts_c
test_inputs = []

for sentence in eval_input_data:
  test_inputs.append(" ".join(sentence))

# #Create a list of targets
eval_output_data = test_tags_c
test_outputs = []

for sentence in eval_output_data:
  test_outputs.append(" ".join(sentence))

In [62]:
# eval_input_sequences
test_inputs[8]

'Or the allometries are not used to extrapolate the biomass of the first item ( for example a tree dbh ) . )'

In [63]:
model.to("cuda" if torch.cuda.is_available() else "cpu")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

The process below predicts targets for 2k data samples in the dev set

In [64]:
predictions = []
references = []

In [65]:
print(test_inputs[47])

Chlamydia caviae infection alters abundance but not composition of the guinea pig vaginal microbiota In humans , the vaginal microbiota is thought to be the first line of defense again pathogens including Chlamydia trachomatis .


In [66]:
# for i in test_inputs:
for i in range(len(test_inputs)):
  print (i)
  sentence = test_inputs[i]
  tokenized_input = tokenizer(sentence, return_tensors='pt', max_length=512)
  input_ids = tokenized_input['input_ids'].to("cuda" if torch.cuda.is_available() else "cpu")
  attention_mask = tokenized_input['attention_mask'].to("cuda" if torch.cuda.is_available() else "cpu")

  try:
        outputs = model.generate(
            input_ids=input_ids, attention_mask=attention_mask, max_length=512
        )
        # print(outputs[0])
        labels = tokenizer.decode(outputs[0], clean_up_tokenization_spaces=True, skip_special_tokens=True)
        # print("prev sentence:", sentence)
        # print("prev_label:", labels)
        sentence = unidecode(sentence).lower()
        labels = unidecode(labels)
        # print("decoded_sentence:", sentence)
        # print("decoded_label:", labels)
        flabels = word_labels(sentence, labels)
        if len(flabels) != len(test_outputs[i].split()):
          print(f"Unequal prediction and reference!")
          continue
        predictions.append(flabels)
        references.append(test_outputs[i].split())

  except Exception as e:
      print(f"Error in iteration {i}: {str(e)}")

0


1
2
3
4
5
6
7
Error in iteration 7: list index out of range
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
Error in iteration 27: list index out of range
28
29
30
31
32
33
Error in iteration 33: list index out of range
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
Error in iteration 65: list index out of range
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
Error in iteration 97: list index out of range
98
99
Error in iteration 99: list index out of range
100
101
102
103
104
Error in iteration 104: list index out of range
105
106
107
108
109
110
111
112
113
Error in iteration 113: list index out of range
114
115
116
117
118
119
120
121
Error in iteration 121: list index out of range
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
Error in iteration 140: list index out of range
141
142
143
144
Error in iteration 144: list index out of range
145
146
147

In [67]:
# List index out of range error
print(test_inputs[8])
print(test_inputs[36])
print(test_inputs[43])

Or the allometries are not used to extrapolate the biomass of the first item ( for example a tree dbh ) . )
117.8998 118.1483 29.2852 29.10178 1/7/2008 ######## no organisms , only vegetated and bare plots no organisms , only vegetated and bare plots tscholten pkuehn cgeissler CSPs : Kinetic energy of raindrops in the CSPs : characteristics of rain events Rain was captured in open field and in vegetated conditions parallel to measurements in the Comparative study plots ( CSPs ) .
We hypothesize that , after extraction of variation that is explained by location and land-use type , soil properties still explain significant proportions of variation in the abundance and diversity of soil biota .


In [68]:
print(test_outputs[8])
print(test_outputs[36])
print(test_outputs[43])

O O O O O O O O O B-Quality O O O O O O O O B-Organism O O O O
O O O O O O O B-Organism O O O O O O O O O O O O O O O O O O O B-Quality I-Quality O B-Matter O O O O O O B-Phenomena I-Phenomena B-Matter O O O O B-Environment O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O B-Environment O O O O O O O O O B-Quality O O O B-Environment O O


In [69]:
for row in references[:10]:
  print(row)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Phenomena', 'I-Phenomena', 'O', 'O', 'B-Environment', 'O', 'O']
['O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'O', 'O', 'B-Organism', 'B-Quality', 'I-Quality', 'O']
['B-Quality', 'I-Quality', 'I-Quality', 'I-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'I-Organism', 'O', 'O', 'B-Organism', 'O', 'O', 'O', 'O']
['O', 'O', 'B-Location', 'I-Location', 'I-Location', 'B-Location', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [70]:
for row in predictions[:10]:
  print(row)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'I-Quality', 'I-Quality', 'O']
['O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-Quality', 'I-Quality', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-Quality', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'B-Organism', 'I-Organism', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['O', 'O', 'O', 'O', 'O

# resultssss

Use seqeval framework introduced to calculate scores F1/precision/recall for every class entity and overall score for model performance

In [71]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

precision = precision_score(references, predictions)
recall = recall_score(references, predictions)
f1 = f1_score(references, predictions)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')

# You can also print a detailed classification report
print(classification_report(references, predictions))

# result = seqeval.compute(predictions=predictions, references=references)
# resultb

Precision: 0.3581
Recall: 0.1470
F1-Score: 0.2085
                precision    recall  f1-score   support

   Environment       0.12      0.01      0.01       140
      Location       0.00      0.00      0.00        20
        Matter       0.00      0.00      0.00        50
      Organism       0.40      0.27      0.32       229
Organism count       0.00      0.00      0.00         0
     Organisms       0.00      0.00      0.00         0
     Phenomena       0.00      0.00      0.00        54
       Quality       0.33      0.19      0.24       262

     micro avg       0.36      0.15      0.21       755
     macro avg       0.11      0.06      0.07       755
  weighted avg       0.26      0.15      0.18       755



/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
print(len(references))

203
